### Importing Libraries

In [19]:
import datetime
import json
import hashlib
import requests
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse

## Creating Block Chain Class

In [20]:
class Blockchain:
    # initializing the Chain Object
    def __init__(self):
        self.chain = [] ### initilzing the chain as empty 
        self.transactions = [] ### no initial transactons
        self.create_block(proof = 1, previous_hash = '0') ## initil block with a proof(Nonce) as 1 and hash as "0"
        self.nodes = set() # empty node list(no ip from different computers initially")
    
    def create_block(self, proof, previous_hash):
        ### initialize block with property 
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'transactions': self.transactions}
        self.transactions = [] ## empty transactions once block has been created
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True
    
    def add_transaction(self, sender, receiver, amount):
        self.transactions.append({'sender': sender,
                                  'receiver': receiver,
                                  'amount': amount})
        previous_block = self.get_previous_block()
        return previous_block['index'] + 1
    
    def add_node(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.path)
    
    def replace_chain(self):
        network = self.nodes
        longest_chain = None
        max_length = len(self.chain)
        for node in network:
            response = requests.get(f'http://{node}/get_chain')
            if response.status_code == 200:
                length = response.json()['length']
                chain = response.json()['chain']
                if length > max_length and self.is_chain_valid(chain):
                    max_length = length
                    longest_chain = chain
        if longest_chain:
            self.chain = longest_chain
            return True
        return False

In [21]:
zion = Blockchain()

In [22]:
app = Flask(__name__)

In [23]:
nodeAddress = str(uuid4()).replace("-", "")

In [24]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousBlock = zion.getLastBlock()
    previousProof = previousBlock["proof"]
    previousHash = zion.hashBlock(previousBlock)
    proof = zion.proofOfWork(previousProof)
    zion.addTransaction(sender = nodeAddress, receiver = "berkat", amount=10)
    newBlock = zion.addBlock(proof, previousHash)
    response = {
        "message": "congrats you have mined a block",
        "index": newBlock["index"],
        "proof": newBlock["proof"],
        "timestamp": newBlock["timestamp"]
    }
    return jsonify(response), 200

In [25]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": zion.chain,
        "length": len(zion.chain)
    }
    return jsonify(response), 200

In [26]:
@app.route("/valid", methods = ["GET"])
def valid():
    isValid = zion.isChainValid(zion.chain)
    if isValid:
        response = {"message": " The chain is valid and can be built on"}
    else:
        response = {"message": "This is not a valid chain"}
    return jsonify(response), 200

In [27]:
@app.route("/connect", methods = ["POST"])
def connect():
    nodeResponse = request.get_json()
    nodes = nodeResponse.get("nodes")
    
    if nodes is None:
        return "there are no nodes here", 400
    for node in nodes:
        zion.addNode(node)
    response = {
        "message": "All nodes are connected",
        "nodes": list(zion.nodes)
    }
    return jsonify(response), 201

In [28]:
@app.route('/connect_node', methods = ['POST'])
def connect_node():
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No node", 400
    for node in nodes:
        zion.add_node(node)
    response = {'message': 'All the nodes are now connected. The Hadcoin Blockchain now contains the following nodes:',
                'total_nodes': list(zion.nodes)}
    return jsonify(response), 201


In [ ]:
app.run(host='0.0.0.0', port = 7000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:7000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jan/2022 20:26:47] "POST /connect_node HTTP/1.1" 201 -
